# Data Analyzes and data in database (DuckDB)

# Data EDA

In [2]:
context

<kedro.framework.context.context.KedroContext object at 0x7f81425dd060>

In [94]:
import unicodedata
import numpy as np
import pandas as pd
import duckdb
import logging

import duckdb
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.feature_selection import SelectPercentile, f_regression, chi2, f_classif
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, r2_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
import logging
from typing import Dict, Tuple

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from kedro.config import ConfigLoader
from kedro.framework.project import settings
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
import functions_project

In [5]:
duckdb

[03/23/23 14:02:08] INFO     Loading data from 'wikiraw' (CSVDataSet)...                        ]8;id=476938;file:///Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=309185;file:///Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [14]:
    ohes = X_train.filter(like='writer_|director_').columns
    cats = X_train.filter(like='origin|genre').columns
    conts = X_train.filter(like='startYear|yeardif|runtimeMinutes|numVotes|winner|emmys|genre|origin|domestic_gross|worldwide_gross').columns


,year,title,origin,genre
0,1901,kansas saloon smashers,American,unknown
1,1901,love by the light of the moon,American,unknown
2,1901,the martyred presidents,American,unknown
3,1901,"terrible teddy, the grizzly king",American,unknown
4,1902,jack and the beanstalk,American,unknown
...,...,...,...,...
34881,2014,the water diviner,Turkish,unknown
34882,2017,calg cengi ikimiz,Turkish,comedy
34883,2017,olanlar oldu,Turkish,comedy
34884,2017,non-transferable,Turkish,romantic comedy


In [129]:
con = duckdb.connect('../data/02_intermediate/imdb_reviews.db')

data = con.execute('''SELECT label,
                           endYear, 
                           primaryTitle,
                           hasoriginalTitle, 
                           endYear - startYear as yeardif, 
                           runtimeMinutes,
                           numVotes,
                           movie_writer.*,
                           movie_directing.*,
                           wiki.*,
                           COALESCE(oscars.winner, -1) AS winner,
                           COALESCE(emmys.win, -1) AS emmys,
                           COALESCE(budget.domestic_gross, -1) AS domestic_gross,
                           COALESCE(budget.worldwide_gross, -1) AS worldwide_gross,
                           COALESCE(budget.production_budget, -1) AS production_budget
                    FROM imdb_review_train
                    LEFT JOIN budget
                      ON (imdb_review_train.endYear >= budget.release_year AND imdb_review_train.startYear <= budget.release_year)
                      AND (imdb_review_train.primaryTitle = budget.movie OR imdb_review_train.originalTitle = budget.movie)
                    LEFT JOIN emmys
                      ON (imdb_review_train.endYear >= emmys.year AND imdb_review_train.startYear <= emmys.year)
                      AND (imdb_review_train.primaryTitle = emmys.nominee OR imdb_review_train.originalTitle = emmys.nominee)
                    LEFT JOIN wiki
                      ON (imdb_review_train.endYear >= wiki.year AND imdb_review_train.startYear <= wiki.year)
                      AND (imdb_review_train.primaryTitle = wiki.title OR imdb_review_train.originalTitle = wiki.title)
                    LEFT JOIN oscars
                      ON (imdb_review_train.endYear >= oscars.year_film AND imdb_review_train.startYear <= oscars.year_film)
                      AND (imdb_review_train.primaryTitle = oscars.film OR imdb_review_train.originalTitle = oscars.film)
                    LEFT JOIN movie_writer
                      ON imdb_review_train.tconst = movie_writer.movie  
                    LEFT JOIN movie_directing
                      ON imdb_review_train.tconst = movie_directing.movie''').fetchdf()

con.close()
duckdb.close()

In [133]:
X_train = data.copy()
y_train = X_train['label']
X_train = X_train.drop('label', axis=1)

In [134]:
parameters = catalog.load('params:model_options')

[03/23/23 21:02:46] INFO     Loading data from 'params:model_options' (MemoryDataSet)...        ]8;id=396446;file:///Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=732398;file:///Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [139]:
X_train = X_train.fillna(-1)

writers = list(X_train.filter(like='writer_').columns)
directors = list(X_train.filter(like='director_').columns)
genre = list(X_train.filter(like='genre_').columns)
origin = list(X_train.filter(like='origin_').columns)
conts = ['endYear', 'yeardif', 'runtimeMinutes', 'numVotes', 'winner', 'emmys', 'domestic_gross', 'worldwide_gross']
varss = writers + directors + genre + origin + conts


In [140]:
X_train = X_train[varss]

In [146]:
df2 = pipeline.fit_transform(X_train)

In [147]:
preprocessor = Pipeline([
    ("selector", ColumnTransformer([
        ("selector", "passthrough", varss)
    ], remainder="drop"))
])

# the models that you want to compare
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'KNeighboursClassifier': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

# the optimisation parameters for each of the above models
params = {
    'RandomForestClassifier': {
        "classifier__n_estimators": [100, 200, 500, 1000],
        "classifier__max_features": [None, "sqrt", "log2"],
        "classifier__bootstrap": [True],
        "classifier__criterion": ['gini', 'entropy'],
        "classifier__oob_score": [True, False]
    },
    'GradientBoostingClassifier': {
        'classifier__n_estimators': [500, 700, 900],
        'classifier__max_depth': [3, 5, 7],
        'classifier__min_samples_split': [15, 25, 50],
        'classifier__max_features': [0.3, 0.6, 0.8],
    },
    # 'GradientBoostingClassifier': {'classifier__learning_rate': [0.1]},

    'KNeighboursClassifier': {
        'classifier__n_neighbors': np.arange(3, 15),
        'classifier__weights': ['uniform', 'distance'],
        'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute']
    },
    'LogisticRegression': {
        'classifier__solver': ['newton-cg', 'sag', 'lbfgs'],
        'classifier__multi_class': ['ovr', 'multinomial']
    }
}

sklearnmodel = parameters['sklearnmodel']
est = models[sklearnmodel]
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", est)]
)

params = {'classifier__learning_rate': [0.05, 0.1]}



clfcv = GridSearchCV(estimator=clf, param_grid=params, cv=4, verbose=3, scoring='accuracy', n_jobs=-1)
clfcv.fit(X_train, y_train.values.ravel())
print(clfcv.best_params_)

Fitting 4 folds for each of 2 candidates, totalling 8 fits
{'classifier__learning_rate': 0.1}
[CV 2/4] END .....classifier__learning_rate=0.1;, score=0.796 total time=  29.0s
[CV 2/4] END ....classifier__learning_rate=0.05;, score=0.786 total time=  29.5s
[CV 3/4] END .....classifier__learning_rate=0.1;, score=0.775 total time=  29.9s
[CV 4/4] END .....classifier__learning_rate=0.1;, score=0.776 total time=  30.0s
[CV 1/4] END .....classifier__learning_rate=0.1;, score=0.780 total time=  30.2s
[CV 3/4] END ....classifier__learning_rate=0.05;, score=0.767 total time=  30.3s
[CV 4/4] END ....classifier__learning_rate=0.05;, score=0.771 total time=  30.4s
[CV 1/4] END ....classifier__learning_rate=0.05;, score=0.773 total time=  30.7s


In [ ]:
clfcv.predict(X_train)

In [19]:
writers + directors

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/x7/py286zss5fl0wvb7b1w8zvlm0000gn/T/ipykernel_35789/2968837484.py:1 in <cell line:  │
│ 1>                                                                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/x7/py286zss5fl0wvb7b1w8zvlm0000gn/T/ipykernel_35789/2968837484.py'                 │
│                                                                                                  │
│ /Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/pandas/c │
│ ore/ops/common.py:72 in new_method                                                               │
│                                                                                                  │
│    69 │   │                                                                                      │
│    70 │   │   other = item_from_zerodim(other)                                                   │
│    71 │   │                                                                                      │
│ ❱  72 │   │   return method(self, other)                                                         │
│    73 │                                                                                          │
│    74 │   return new_method                                                                      │
│    75                                                                                            │
│                                                                                                  │
│ /Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/pandas/c │
│ ore/arraylike.py:102 in __add__                                                                  │
│                                                                                                  │
│    99 │                                                                                          │
│   100 │   @unpack_zerodim_and_defer("__add__")                                                   │
│   101 │   def __add__(self, other):                                                              │
│ ❱ 102 │   │   return self._arith_method(other, operator.add)                                     │
│   103 │                                                                                          │
│   104 │   @unpack_zerodim_and_defer("__radd__")                                                  │
│   105 │   def __radd__(self, other):                                                             │
│                                                                                                  │
│ /Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/pandas/c │
│ ore/indexes/base.py:7050 in _arith_method                                                        │
│                                                                                                  │
│   7047 │   │   │   # See https://github.com/pandas-dev/pandas/issues/31109                       │
│   7048 │   │   │   return NotImplemented                                                         │
│   7049 │   │                                                                                     │
│ ❱ 7050 │   │   return super()._arith_method(other, op)                                           │
│   7051 │                                                                                         │
│   7052 │   @final                                                                                │
│   7053 │   def _unary_method(self, op):                                                          │
│                                                            

In [13]:
ohes

Index([], dtype='object')

In [35]:
# Transformers
X_train = data.fillna(0)
cats, conts = get_columns(X_train)

data = data.fillna(0)
cats, conts = get_columns(data)
X = data[cats + conts]
y = data["label"]

In [32]:
parameters = catalog.load('params:model_options')

[03/23/23 13:29:20] INFO     Loading data from 'params:model_options' (MemoryDataSet)...        ]8;id=262251;file:///Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=864522;file:///Users/janjaniszewski/opt/anaconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [33]:
sklearnmodel = parameters['sklearnmodel']
est = models[sklearnmodel]
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", est)]
)

# ohe = AdvancedOneHotEncoder()
# outs = ohe.fit(X_train[cats], y_train)
# outs = ohe.transform(X_train[cats])

params = {'classifier__learning_rate': [0.05, 0.1]}


In [41]:
#clf.fit(X, y.values.ravel())
clfcv = GridSearchCV(estimator=clf, param_grid=params, cv=2, verbose=3, scoring='accuracy', n_jobs=-1)
clfcv.fit(X, y.values.ravel())

Fitting 2 folds for each of 2 candidates, totalling 4 fits


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['startYear',
                                                                          'yeardif',
                                                                          'runtimeMinutes',
                                                                          'numVotes',
                                                                          'hasoriginalTitle',
                                                                          'winner',
                                                                          'emmys']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('selector',
                                                                                          SelectPercentile(percentile=20,
                                                                                                           score_func=<function chi2 at 0x7fb28c0a7250>))]),
                                                                         ['wr...
                                                                          'writer_nm0000180',
                                                                          'writer_nm0000184',
                                                                          'writer_nm0000186',
                                                                          'writer_nm0000217',
                                                                          'writer_nm0000219',
                                                                          'writer_nm0000229',
                                                                          'writer_nm0000230',
                                                                          'writer_nm0000231',
                                                                          'writer_nm0000233',
                                                                          'writer_nm0000241',
                                                                          'writer_nm0000247', ...])])),
                                       ('classifier',
                                        GradientBoostingClassifier())]),
             n_jobs=-1, param_grid={'classifier__learning_rate': [0.05, 0.1]},
             scoring='accuracy', verbose=3)

In [42]:
clfcv.predict(X_train)

array([ True, False, False, ..., False, False, False])

In [48]:
con.close()
duckdb.close()

In [18]:








clfcv = GridSearchCV(estimator=clf, param_grid=params, cv=4, verbose=3, scoring='accuracy', n_jobs=-1)
print(clfcv.best_params_)
return clfcv

,startYear,yeardif,primaryTitle,hasoriginalTitle,label,runtimeMinutes,numVotes,writer_nm0000005,writer_nm0000019,writer_nm0000033,...,director_nm7625681,director_nm7834624,director_nm7960857,director_nm7992231,director_nm8448178,director_nm9227039,director_nm9284362,movie_2,winner,emmys
0,2001,0,friends,1,True,167.0,58873.233010,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt0308280,-1,0
1,2007,0,ghosts of abu ghraib,1,True,78.0,1017.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt0912585,-1,1
2,2013,0,richard pryor: omit the logic,1,True,83.0,1595.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt2401223,-1,0
3,1936,0,swing time,1,True,103.0,19143.882353,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt0028333,1,-1
4,1939,0,"goodbye, mr. chips",1,True,114.0,10311.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt0031385,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7954,2019,0,18am padi,1,False,160.0,1321.000000,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt7692434,-1,-1
7955,2018,0,haami,1,True,133.0,1185.000000,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt8364356,-1,-1
7956,2018,0,burn the stage: the movie,1,True,84.0,2751.000000,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt9151704,-1,-1
7957,1988,0,"thelonious monk: straight, no chaser",0,True,90.0,1193.000000,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tt0098465,-1,-1


In [29]:
dummie_all_director

,movie,director_nm0000005,director_nm0000008,director_nm0000009,director_nm0000019,director_nm0000033,director_nm0000036,director_nm0000037,director_nm0000040,director_nm0000041,...,director_nm9792122,director_nm9798074,director_nm9813479,director_nm9818807,director_nm9825418,director_nm9902054,director_nm9942830,director_nm9955258,director_nm9958352,director_nm9985316
0,tt0003740,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0008663,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0009369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0010307,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0010600,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,tt9850344,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9994,tt9850386,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9995,tt9900782,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,tt9904802,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Import necessary libraries
import pandas as pd
import glob
from ydata_profiling import ProfileReport
import numpy as np
import json
import duckdb
import string
import re
import unicodedata

In [136]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 2, 0, 2], [0, 1, 4, 3], [0, 1, 1, 3]]
selector = VarianceThreshold()
selector.fit_transform(X)

array([[2, 0, 2],
       [1, 4, 3],
       [1, 1, 3]])

In [42]:
con = duckdb.connect('../data/02_intermediate/imdb_reviews.db')

IOException: IO Error: Could not set lock on file "/Users/janjaniszewski/Documents/University/project-Big-Data/pipeline/notebooks/../data/02_intermediate/imdb_reviews.db": Resource temporarily unavailable

In [132]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectPercentile, chi2
X, y = load_digits(return_X_y=True)
X_new = SelectPercentile(chi2, percentile=10).fit_transform(X, y)

In [134]:
X_new

array([[ 8.,  5.,  8., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ..., 16.,  0.,  0.],
       [ 0.,  1.,  8., ..., 16.,  5.,  9.],
       ...,
       [ 0.,  0.,  9., ..., 14.,  0.,  0.],
       [ 4.,  0.,  5., ...,  0.,  2.,  0.],
       [ 0.,  0., 12., ...,  6.,  8.,  1.]])

In [126]:
labels = pd.read_csv('../../data/train-1.csv')
directors = pd.read_json('../../data/directing.json')
writers = pd.read_json('../../data/writing.json')

writers = writers.replace('\\N', np.nan)
directors = directors.replace('\\N', np.nan)


labels = labels[['label', 'tconst']]
writers_directors = pd.merge(writers, directors, on='movie')
famousness = pd.merge(labels, writers_directors, left_on='tconst', right_on='movie')

famousness_writer = famousness.groupby('writer', as_index=False)[['label']].mean()
famousness_directors = famousness.groupby('director', as_index=False)[['label']].mean()


famousness_writer.columns = ['writer' , 'famous']
famousness_directors.columns = ['director', 'famous']

famousness_writer.loc[famousness_writer['famous'] > 0, 'famous'] = 1
famousness_directors.loc[famousness_directors['famous'] > 0, 'famous'] = 1

famousness = pd.merge(famousness, famousness_writer, on='writer')
famousness = pd.merge(famousness, famousness_directors, on='director', suffixes=('_writer', '_director'))

return famousness.groupby('tconst', as_index=False)[['famous_writer', 'famous_director']].max()

In [129]:
df1 = famousness.groupby('tconst', as_index=False)[['famous_writer', 'famous_director']].max()

In [131]:
df1['famous_director'].mean()

0.536663124335813

In [52]:
def dataeda_storage(movie_team):
    movie_team = movie_team.replace('\\N', np.nan)
    # Remove movies with missing writer (= 297 movies)
    movie_team = movie_team.dropna()

    title = movie_team.columns[1]

    # Drop irrelevant team members
    counts = movie_team.groupby('writer', as_index=False).count()
    relevants = list(counts['writer'][counts['movie'] >= 2])


    movie_team = movie_team[movie_team['writer'].isin(relevants)]

    counts = movie_team.groupby('movie').count().reset_index()
    counts.columns = ['movie', f'count_of_{title}']



    # Transform data and remove outliers before pushing data into the database
    # Select first 8 writers (only <1% of the movies have more than 8 writers, see notebook 1.B)
    # Transform data so that the writers are grouped per movie.
    # Reset index so that every column is added to database
    top_movie_team = movie_team.groupby(['movie'], as_index=False).agg(
        {title: ', '.join})
    movie_and_writers = pd.concat(
        [top_movie_team['movie'], top_movie_team[title].str.split(', ', expand=True)], axis=1)
    repeatcount = len(movie_and_writers.columns) - 1
    movie_and_writers.columns = ['movie'] + ([title]*repeatcount)
    movies = movie_and_writers['movie']
    ohe = pd.get_dummies(movie_and_writers.drop('movie', axis=1))
    movie_and_writers = ohe.groupby(level=0, axis=1).sum()
    movie_and_writers['movie'] = movies
    movie_and_writers = pd.merge(movie_and_writers, counts, on='movie')

    print('Before', len(movie_and_writers.columns))
    for col in movie_and_writers.drop('movie', axis=1).columns:
        if movie_and_writers[col].sum() <= 1:
            movie_and_writers.drop(col, inplace=True, axis=1)
    print('After', len(movie_and_writers.columns))

    return movie_and_writers

In [35]:
df = con.execute(sl).fetchdf()

In [75]:
def dataeda_storage(movie_team):
    movie_team = movie_team.replace('\\N', np.nan)
    # Remove movies with missing writer (= 297 movies)
    movie_team = movie_team.dropna()

    title = movie_team.columns[1]

    counts = movie_team.groupby('movie').count().reset_index()
    counts.columns = ['movie', f'count_of_{title}']

    # Transform data and remove outliers before pushing data into the database
    # Select first 8 writers (only <1% of the movies have more than 8 writers, see notebook 1.B)
    # Transform data so that the writers are grouped per movie.
    # Reset index so that every column is added to database
    top_movie_team = movie_team.groupby(['movie'], as_index=False).agg(
        {title: ', '.join})
    movie_and_writers = pd.concat(
        [top_movie_team['movie'], top_movie_team[title].str.split(', ', expand=True)], axis=1)
    repeatcount = len(movie_and_writers.columns) - 1
    movie_and_writers.columns = ['movie'] + ([title]*repeatcount)
    movies = movie_and_writers['movie']
    ohe = pd.get_dummies(movie_and_writers.drop('movie', axis=1))
    movie_and_writers = ohe.groupby(level=0, axis=1).sum()
    movie_and_writers['movie'] = movies
    movie_and_writers = pd.merge(movie_and_writers, counts, on='movie')

    counts = movie_and_writers.drop('movie', axis=1).apply(sum, 0)
    relevant_cols = list(counts[counts >= 1].keys()) + ['movie']
    movie_and_writers = movie_and_writers[relevant_cols]

    return movie_and_writers

In [76]:
dataeda_storage(df)

,writer_nm0000005,writer_nm0000019,writer_nm0000027,writer_nm0000033,writer_nm0000036,writer_nm0000040,writer_nm0000041,writer_nm0000045,writer_nm0000051,writer_nm0000059,...,writer_nm9925241,writer_nm9933959,writer_nm9942830,writer_nm9946633,writer_nm9958353,writer_nm9980769,writer_nm9985316,writer_nm9985837,count_of_writer,movie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,tt0003740
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,tt0008663
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,tt0009369
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,tt0010307
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,tt0010600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,tt9850344
9699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,tt9850386
9700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,tt9900782
9701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,tt9904802


In [7]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
translator.detect('Ich bin Deutsch')

AttributeError: 'NoneType' object has no attribute 'group'

In [41]:
movie_and_writers = df.copy()
movie_and_writers.groupby('movie').count().reset_index()
movie_and_writers.columns = ['movie', f'{title}_count']

,movie,writer
0,tt0003740,4
1,tt0008663,3
2,tt0009369,2
3,tt0010307,1
4,tt0010600,4
...,...,...
9995,tt9850344,3
9996,tt9850386,1
9997,tt9900782,2
9998,tt9904802,2


In [101]:
aohe = AdvancedOneHotEncoder()

aohe.fit_transform(df)

,movieID,primaryTitle,year,runtimeMinutes,numVotes,rating,genre_1,genre_2,genre_3,genre_4,...,director_nm2639032,director_nm3418586,director_nm3809228,director_nm3952461,director_nm4335102,director_nm6438942,director_nm8933359,director_nan,label_False,label_True
0,tt0112453,balto,1995,78.0,42057.00000,2.624566,Adventure,Animation,Children,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,tt0113987,nixon,1995,192.0,30228.00000,3.333528,Drama,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,tt0114388,sense and sensibility,1995,136.0,54040.46988,3.823707,Drama,Romance,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,tt0113845,money train,1995,110.0,40354.00000,2.642168,Action,Comedy,Crime,Drama,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,tt0113627,leaving las vegas,1995,111.0,120030.00000,3.187584,Drama,Romance,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7954,tt9351980,american factory,2019,110.0,20441.00000,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
7955,tt2215151,inequality for all,2013,89.0,6793.00000,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
7956,tt0497646,"nou, dat was het dan!",2005,80.0,2151.00000,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
7957,tt3665486,los scandalous - skid row,2014,100.0,1374.00000,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [100]:
class AdvancedOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, method='advanced', handle_unknown="infrequent_if_exist", sparse_output=False):
        self.method = method
        self.ohe = OneHotEncoder(sparse_output=sparse_output, handle_unknown=handle_unknown)

    def fit(self, X, y=None):
        self.ohe.fit(X)
        return self

    def transform(self, X, y=None):
        if self.method == 'advanced':
            X.columns = ['director' if 'director' in colname else colname for colname in df.columns]
            X.columns = ['writer' if 'writer' in colname else colname for colname in df.columns]
            transformed = self.ohe.transform(X)

            # Create a Pandas DataFrame of the hot encoded column
            ohe_df = pd.DataFrame(transformed, columns=self.ohe.get_feature_names_out())
            ohe_df2 = ohe_df.groupby(level=0, axis=1).sum()
            X = pd.concat([X, ohe_df], axis=1).drop(['writer', 'director'], axis=1)
        else:
            Exception('Method: Simple not impmlemented yet')
        return X

,writer_nan,writer_nm0000005,writer_nm0000019,writer_nm0000027,writer_nm0000033,writer_nm0000036,writer_nm0000040,writer_nm0000041,writer_nm0000051,writer_nm0000067,...,writer_nm9920219,writer_nm9920220,writer_nm9922522,writer_nm9925241,writer_nm9933959,writer_nm9942830,writer_nm9946633,writer_nm9958353,writer_nm9985316,writer_nm9985837
0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7954,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7955,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7956,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7957,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
counts = df.groupby(['writer_1', 'label']).count()[['movieID']].sort_values('movieID', ascending=False).reset_index()

counts['success'] = counts['movieID'] * counts['label']
counts.groupby('writer_1').

,writer_1,label,movieID,success
0,nm0000095,True,13,13
1,nm0000005,True,9,9
2,nm0006445,True,8,8
3,nm0408488,True,8,8
4,nm0000264,True,8,8
...,...,...,...,...
6448,nm0575963,True,1,1
6449,nm0026442,True,1,1
6450,nm0574966,False,1,0
6451,nm0574110,True,1,1


In [33]:
imdb_review = pd.read_csv('../../data/test_hidden.csv')
imdb_review = imdb_review.replace('\\N', np.nan)
imdb_review['originalTitle'] = imdb_review['originalTitle'].fillna(imdb_review['primaryTitle'])
imdb_review['endYear'] = imdb_review['endYear'].fillna(imdb_review['startYear']).astype(int)
imdb_review['startYear'] = imdb_review['endYear'].fillna(imdb_review['endYear']).astype(int)
imdb_review['runtimeMinutes'] = imdb_review['runtimeMinutes'].astype(float)
fillers = imdb_review.dropna().groupby('startYear')[['runtimeMinutes', 'numVotes']].mean().reset_index()
imdb_review = pd.merge(imdb_review, fillers, how='left', on='startYear', suffixes=('', "_filler"))
imdb_review['numVotes'] = imdb_review['numVotes'].fillna(imdb_review['numVotes_filler'])
imdb_review['numVotes'] = imdb_review['numVotes'].fillna(imdb_review['numVotes'].mean())
imdb_review['runtimeMinutes'] = imdb_review['runtimeMinutes'].fillna(imdb_review['runtimeMinutes_filler'])
imdb_review['runtimeMinutes'] = imdb_review['runtimeMinutes'].fillna(imdb_review['runtimeMinutes'].mean())
imdb_review['originalTitle'] = imdb_review['originalTitle'].apply(
    lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode()).str.lower()
imdb_review['primaryTitle'] = imdb_review['primaryTitle'].apply(
    lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode()).str.lower()
imdb_review = imdb_review.drop(columns=['Unnamed: 0', 'numVotes_filler', 'runtimeMinutes_filler'])
assert all(imdb_review.apply(lambda x: len(x.dropna()) / len(x)) == 1), 'Empty values found after preprocessing'

len(imdb_review)

1086

In [37]:
df2 = con.execute('select * from imdb_review_validation').fetchdf()
len(df2)

955

In [44]:
list(df2.dtypes[df2.dtypes == 'object'].keys())

['tconst', 'primaryTitle', 'originalTitle']

In [47]:
con.close()
duckdb.close()


In [99]:
con.execute('select * from directors_intermediate').fetchdf()
con.close()

In [66]:
imdb_review.dtypes

primaryTitle       object
originalTitle      object
startYear          object
endYear            object
runtimeMinutes     object
numVotes          float64
label                bool
dtype: object

In [100]:
con.close()
duckdb.close()

In [56]:
imdb_review = pd.read_csv('../../data/validation_hidden.csv', index_col=0)
imdb_review = imdb_review
imdb_review = imdb_review.replace('\\N', np.nan)

imdb_review.apply(lambda x: len(x.dropna()) / len(x))

tconst            1.000000
primaryTitle      1.000000
originalTitle     0.509948
startYear         0.901571
endYear           0.098429
runtimeMinutes    0.997906
numVotes          0.904712
dtype: float64

In [35]:
df3 = pd.read_csv('../../data/test_hidden.csv')

In [37]:
df1

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True
...,...,...,...,...,...,...,...,...,...
958,9955,tt9558612,PM Náréndrá Mớdi,PM Narendra Modi,2019,\N,136,7005.0,False
959,9960,tt9598172,Sáving Léningrád,NaN,2019,\N,96,2200.0,False
960,9977,tt9691136,Shadow in the Cloud,NaN,2020,\N,83,22617.0,False
961,9979,tt9695258,So My Grandma's a Lesbian!,Salir del ropero,2019,\N,94,1054.0,False


In [27]:
valid[['year', 'runtimeMinutes', 'numVotes']].apply(lambda x: len(x.dropna()) / len(x))

year              0.948691
runtimeMinutes    0.997906
numVotes          0.904712
dtype: float64

In [28]:
test[['year', 'runtimeMinutes', 'numVotes']].apply(lambda x: len(x.dropna()) / len(x))

year              0.945672
runtimeMinutes    0.999079
numVotes          0.890424
dtype: float64

In [29]:
train[['year', 'runtimeMinutes', 'numVotes']].apply(lambda x: len(x.dropna()) / len(x))

year              1.000000
runtimeMinutes    1.000000
numVotes          0.900705
dtype: float64

# EDA imdb data

In [3]:
# Read imdb labaled train data
files = glob.glob(r'data/train*.csv')
imdb_review = pd.concat((pd.read_csv(file, on_bad_lines='skip', index_col=1) for file in files))
imdb_review = imdb_review.drop(columns=['Unnamed: 0'])

# Replace all missing values by legit missing values instead of string "\N"
imdb_review = imdb_review.replace('\\N', np.nan)

# Remove 13 rows (< 1% of all rows) where runtimeMinutes is nan
imdb_review = imdb_review[imdb_review['runtimeMinutes'].notna()]
imdb_review['runtimeMinutes'] = imdb_review['runtimeMinutes'].astype(int)

#replace startYear NA with the value of endYear
imdb_review['startYear']=imdb_review['startYear'].fillna(imdb_review['endYear'])
#imdb_review['startYear'].isna().sum()

# Reset index so that every column is added to database
imdb_review = imdb_review.reset_index()
imdb_review


,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,tt0010600,The Doll,Die Puppe,1919,NaN,66,1898.0,True
1,tt0011841,Way Down East,Way Down East,1920,NaN,145,5376.0,True
2,tt0012494,Déstiny,Der müde Tod,1921,NaN,97,5842.0,True
3,tt0015163,The Navigator,The Navigator,1924,NaN,59,9652.0,True
4,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,NaN,93,17887.0,True
...,...,...,...,...,...,...,...,...
7941,tt9625664,Trauma Center,NaN,2019,NaN,87,12951.0,False
7942,tt9741310,Slaxx,Slaxx,2020,NaN,77,2464.0,False
7943,tt9742392,Kindred,Kindred,2020,NaN,101,1719.0,False
7944,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,NaN,111,4144.0,True


In [4]:
def my_map(function_to_apply, list_of_inputs):
    """
    Traverses a list, apply some function to each of the elements and collects the output.
    """
    
    new_list = []
    for i in list_of_inputs:
        new_list.append(function_to_apply(i))
        
    return new_list

In [5]:
def convert_word(word):
    """
    Strips a word from all punctuation, whitespace, and digits. Then converts the word into all lower case
    and convert all characters with accents to non-accent letters.
    """
    word = re.sub(r'[^\w\s]','',word).lower()
    word = re.sub(r'[àáâãäå]', 'a', word)
    word = re.sub(r'[èéêë]', 'e', word)
    word = re.sub(r'[ìíîï]', 'i', word)
    word = re.sub(r'[òóôõö]', 'o', word)
    word = re.sub(r'[ùúûü]', 'u', word)
    
    return word

In [6]:
def map_cleaned_strings(dfname, columnname):
    """
    Maps new values in a df-column. From a new created dictionary, with as Key the 'old' string value and 
    as Value the 'new'-string value. The 'new'-string values are extracted with the function convert_word().
    """
    dict_old_new = {}
    for i in dfname[columnname]:
        if type(i) != str:
            continue
        else:
            dict_old_new[i] = convert_word(i)

    dfname[columnname] = dfname[columnname].map(dict_old_new)
    
    return dfname[columnname]

In [7]:
#call function map_cleaned_strings to apply in the dataframe on the column primary title
map_cleaned_strings(imdb_review, 'primaryTitle')
#...I will work further here after lunch


0                                           the doll
1                                      way down east
2                                            destiny
3                                      the navigator
4                           the phantom of the opera
                            ...                     
7941                                   trauma center
7942                                           slaxx
7943                                         kindred
7944    the bee gees how can you mend a broken heart
7945                             the marriage escape
Name: primaryTitle, Length: 7946, dtype: object

In [8]:
#check if - is also out
for i in imdb_review['primaryTitle']:
    if '-' in i:
        print(i)

In [9]:
# Perform EDA report on imdb reviews
profile = ProfileReport(imdb_review, title="Report")
profile

# EDA remarks
# remove original title? >50% missing values
# remove endYear? >90% missing values
# some titles are duplicate, problem? I don't think so
# what about outliers (number of votes)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# highly rated movies mostly have more runtime minutes and have more number of votes
imdb_review.groupby('label').mean()

,runtimeMinutes,numVotes
label,,
False,97.986872,10053.794026
True,113.341029,48761.503880


## read json files

In [11]:
# Load mapping table movie to writer
movie_writer = pd.read_json('data/writing.json')

# Replace missing values by legit missing values
movie_writer = movie_writer.replace('\\N', np.nan)

# Create an EDA report
profile = ProfileReport(movie_writer, title="Report")
profile

# EDA remarks
# Some movies have multiple writers. Movie tt0401711 even has 29 writers. How are we going to map this? 
# What to do with rows with missing writer value? I think we should remove those rows.

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# Load mapping table movie to director
directing = pd.read_json('data/directing.json')

# Replace missing values by legit missing values
directing = directing.replace('\\N', np.nan)

# Create an EDA report
profile = ProfileReport(directing, title="Report")
profile

# # EDA remarks
# # Some movies have multiple directors. Movie tt1687247 even has 35 writers. How are we going to map this? 
# # What to do with rows with missing director value? I think we should remove those rows.

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
# Remove movies with missing director (= 2 movies)
directing = directing.dropna()

# Transform data and remove outliers before pushing data into the database
# Select first 5 directors (only <1% of the movies have more than 5 directors, see notebook 1.A)
# Transform data so that the directors are grouped per movie.
# Reset index so that every column is added to database
top_5_directors_per_movie = directing.groupby('movie').head(5).groupby(['movie'], as_index=False).agg({'director': ', '.join})
movies_and_directors = pd.concat([top_5_directors_per_movie['movie'], top_5_directors_per_movie['director'].str.split(', ', expand=True)], axis=1)
movies_and_directors = movies_and_directors.rename(columns={0: "director_1", 1: "director_2", 2: "director_3", 3: "director_4", 4: "director_5"})


# Remove movies with missing writer (= 297 movies)
movie_writer = movie_writer.dropna()

# Transform data and remove outliers before pushing data into the database
# Select first 8 writers (only <1% of the movies have more than 8 writers, see notebook 1.B)
# Transform data so that the writers are grouped per movie.
# Reset index so that every column is added to database
top_8_writers_per_movie = movie_writer.groupby('movie').head(8).groupby(['movie'], as_index=False).agg({'writer': ', '.join})
movie_and_writers = pd.concat([top_8_writers_per_movie['movie'], top_8_writers_per_movie['writer'].str.split(', ', expand=True)], axis=1)
movie_and_writers = movie_and_writers.rename(columns={0: "writer_1", 1: "writer_2", 2: "writer_3", 3: "writer_4", 4: "writer_5", 5: "writer_6", 6: "writer_7", 7: "writer_8"})


In [14]:
#dummie tables
# get top 10 most frequent names for director and writer 

##director
n = 5
inlist = directing['director'].value_counts()[:n].index.tolist()

directing_top_5_most = directing[directing[['director']].isin(inlist).any(1)]
#dummie table
dummie_director = pd.get_dummies(directing_top_5_most, columns=['director'])
#rename columns
dummie_director = dummie_director.set_axis(['movie','director_1_dum', 'director_2_dum', 'director_3_dum', 'director_4_dum', 'director_5_dum'], axis=1, inplace=False)

##writer
n = 5
inlist = movie_writer['writer'].value_counts()[:n].index.tolist()

writer_top_5_most = movie_writer[movie_writer[['writer']].isin(inlist).any(1)]
#dummie table
dummie_writer = pd.get_dummies(writer_top_5_most, columns=['writer'])
#rename columns
dummie_writer = dummie_writer.set_axis(['movie','writer_1_dum', 'writer_2_dum', 'writer_3_dum', 'writer_4_dum', 'writer_5_dum'], axis=1, inplace=False)
dummie_writer

,movie,writer_1_dum,writer_2_dum,writer_3_dum,writer_4_dum,writer_5_dum
184,tt0018526,0,0,0,1,0
384,tt0023940,0,0,0,1,0
438,tt0024481,0,0,0,1,0
699,tt0029442,0,0,0,1,0
802,tt0031381,0,0,0,1,0
...,...,...,...,...,...,...
19641,tt4476736,0,1,0,0,0
20500,tt5662932,0,0,0,0,1
20822,tt6129302,0,0,0,0,1
20992,tt6409416,0,0,0,0,1


# DuckDB
#### Why database system?
1. Loading data real time
2. Pandas very poor query engine
3. No query optimizer
4. No support for out of memory computation
5. No support for parallelization
6. Unoptimized implementations for join/aggregations

Data scientists need the functionality database systems offer, but they opt not to use database systems. This often leads to problems down the road when the data gets bigger..

#### Combining Database Systems with Data Science
1. Database Client Connections: DBMS is separate process, queries & data transferrred over socket. Problem is that data transfer is very slow and requires setup & management of DBMS server.
2. User Defined Functions: most cases a data scientist exports data from the RDBMS to the analytical tool. Analytics is run inside DBMS server. No separate analytics program. Problems: difficult to implement and debug, DBMS-specific, requires knowledge of DBMS internals, also requires setup & mangement of DBMS server.
3. Embedded Database Systems: It runs inside the analytics applications, has same low-transfer cost advantages as UDFs. Are easy to install/use. Optimized for many small transcations, not perse for analytical goals. SQLite most famous one.


#### Why DuckDB (and not Spark)?

Avoid using distributed systems (Like Spark) whenever you can, but use RDBMS instead! 

Pro's
1. DuckDB: The SQLite (= embedded database systems) for Analytics. Difference: SQLite uses a row-storage model, where DuckDB uses a columnar storage model. Faster loading for analytical goals.
2. Simple installation 
3. Embedded: no server management
4. Single file storage format
5. Fast analytical processing
6. Fast transfer between R/Python and RDBMS

Made by DB researchers: Column-storage, vectorized processing, MVCC, ART index, Unnesting arbitrary queries.



In [15]:
# create a connection to a file called 'file.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS imdb_review_train AS SELECT * FROM imdb_review")
con.sql("CREATE TABLE IF NOT EXISTS movie_writer AS SELECT * FROM movie_and_writers")
con.sql("CREATE TABLE IF NOT EXISTS movie_directing AS SELECT * FROM movies_and_directors")
con.sql("CREATE TABLE IF NOT EXISTS dummie_director AS SELECT * FROM dummie_director")
con.sql("CREATE TABLE IF NOT EXISTS dummie_writer AS SELECT * FROM dummie_writer")

con.close()